In [1]:
!pip install transformers datasets peft torch

/bin/bash: /home/agus/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [3]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict

/media/agus/Disk 2/projects/subin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_checkpoint = 'meta-llama/Meta-Llama-3-8B-Instruct'
data_path = "ritsu_bot/markdown_files"

In [5]:
from transformers import BitsAndBytesConfig

In [6]:
# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(
        load_in_8bit=True
)

In [7]:
# Load model, tokenizer, and configuration
model_config = AutoConfig.from_pretrained(model_checkpoint, trust_remote_code=True, max_new_tokens=1024)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, 
                quantization_config=bnb_config,
                device_map="auto", trust_remote_code=True, config=model_config)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.18s/it]


In [8]:
!pip install bitsandbytes setuptools

/bin/bash: /home/agus/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [9]:
# Load dataset
dataset = load_dataset('text', data_files={'train': f'{data_path}/*.md'}, split='train')
#def tokenize_function(examples):
#    return tokenizer(examples["text"], padding="max_length", truncation=True)
#tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = dataset.map(
    lambda examples: {
        "input_ids": tokenizer(examples["text"], truncation=True, padding="max_length")["input_ids"],
        "attention_mask": tokenizer(examples["text"], truncation=True, padding="max_length")["attention_mask"],
        "labels": tokenizer(examples["text"], truncation=True, padding="max_length")["input_ids"]  # Same as input_ids
    },
    batched=True
)




Map:   0%|          | 0/12337 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 12337/12337 [00:00<00:00, 21790.28 examples/s]


In [10]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12337
})

In [11]:
# Configure LoRA
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,  
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj']
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='no',  # No evaluation
    per_device_train_batch_size=1,  # Small batch size
    gradient_accumulation_steps=16,  # Accumulate gradients for 16 steps
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    fp16=True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

/media/agus/Disk 2/projects/subin/.venv/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Train the model
trainer.train()

  0%|          | 10/2313 [00:50<3:09:26,  4.94s/it]

{'loss': 3.2108, 'grad_norm': 1.6553657054901123, 'learning_rate': 4.980544747081712e-05, 'epoch': 0.01}


  1%|          | 20/2313 [01:40<3:08:59,  4.95s/it]

{'loss': 3.3869, 'grad_norm': 2.9521872997283936, 'learning_rate': 4.963251188932123e-05, 'epoch': 0.03}


  1%|▏         | 30/2313 [02:27<2:59:18,  4.71s/it]

...

100%|█████████▉| 2310/2313 [2:59:58<00:13,  4.60s/it]

{'loss': 1.6549, 'grad_norm': 7.2301025390625, 'learning_rate': 1.729355814958928e-07, 'epoch': 3.0}


100%|██████████| 2313/2313 [3:00:13<00:00,  4.68s/it]

{'train_runtime': 10813.8489, 'train_samples_per_second': 3.423, 'train_steps_per_second': 0.214, 'train_loss': 2.0878401649910496, 'epoch': 3.0}


TrainOutput(global_step=2313, training_loss=2.0878401649910496, metrics={'train_runtime': 10813.8489, 'train_samples_per_second': 3.423, 'train_steps_per_second': 0.214, 'total_flos': 3.372059720503296e+16, 'train_loss': 2.0878401649910496, 'epoch': 2.9997568290508227})

In [14]:
# Save the model with LoRA parameters
model.save_pretrained('./finetuned_model', state_dict=get_peft_model_state_dict(model))

# Save the tokenizer
tokenizer.save_pretrained('./finetuned_model')

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/tokenizer.json')